# DEMO: Human Development Index (HDI)

## CURARE data collection model

#### Type your dataset's URL or leave default value

In [1]:
import ipywidgets as widgets

In [69]:
#url_tb = widgets.Text(value='http://ergast.com/downloads/f1db_csv.zip', description='Dataset url:', disabled=False); url_tb
url_tb = widgets.Text(value='http://hdr.undp.org/en/indicators/137506#', description='Dataset url:', disabled=False); url_tb

Text(value='http://hdr.undp.org/en/indicators/137506#', description='Dataset url:')

#### Type additional information about your dataset or leve default values

In [2]:
dbprovider_tb = widgets.Text(value='United Nations Development Programme', description='provider:', disabled=False); dbprovider_tb

Text(value='United Nations Development Programme', description='provider:')

In [3]:
dblicense_tb = widgets.Text(value='Creative Commons', description='license:', disabled=False); dblicense_tb

Text(value='Creative Commons', description='license:')

In [4]:
dbauthor_tb = widgets.Text(value='ergast', description='author:', disabled=False); dbauthor_tb

Text(value='ergast', description='author:')

In [5]:
dbdesc_tb = widgets.Text(value='Data for the Formula One series, from the beginning of the world championships in 1950.', 
                            description='description:', disabled=False); dbdesc_tb

Text(value='Data for the Formula One series, from the beginning of the world championships in 1950.', descript…

## Data Collection Class

In [81]:
class DataCollection:
    _id = ""
    name = ""
    provider = ""
    licence = ""
    size = 0
    author = ""
    description = ""        
    
    releaseList = []
 
    def __init__(self, _id, name, provider, licence, size, author, description, releaseList):
        self._id = _id
        self.name = name
        self.provider = provider
        self.licence = licence
        self.size = size
        self.author = author
        self.description = "Add API decription"
        
        self.releaseList = releaseList
 
    def printInfo(self):
        print ("_id  = " + str(self._id))
        print ("name  = " + str(self.name))
        print ("provider  = " + str(self.provider))
        print ("licence  = " + str(self.licence))
        print ("size = " + str(self.size) + " Bytes")
        print ("author  = " + str(self.author))
        print ("description  = " + str(self.description))
        print ("list of releases = " + str(self.releaseList))      
        
print ("Success class created!")

Success class created!


## Release Class

In [80]:
# Get conditions in which data is produced
class Release:
    _id = ""
    releaseNum = 0
    publicationDate = ""    
    
    itemList = []
    size = 0   # added by me
    
    def __init__(self, _id, releaseNum, publicationDate, itemList, size):
        self._id = _id
        self.releaseNum = releaseNum
        self.publicationDate = publicationDate        
        self.itemList = itemList
        self.size = size
        
    def incRelease(self):
        self.releaseNum = self.releaseNum + 1    
        
    def printInfo(self):
        print ("_id  = " + str(self._id))
        print ("releaseNum  = " + str(self.releaseNum))
        print ("publicationDate  = " + str(self.publicationDate))                
        print ("List of items = " + str(self.itemList))
        print ("size = " + str(self.size) + " Bytes")

print ("Success class created!")

Success class created!


## Item Class

In [79]:
class Item:
    _id = ""
    name = ""
    content = []
    
    size = 0   # added by me
 
    def __init__(self, _id, name, content, size):
        self._id = _id
        self.name = name
        self.content = content
        self.size = size
        
    def printInfo(self):
        print ("_id  = " + str(self._id))
        print ("name  = " + str(self.name))
        print ("content  = " + str(self.content))
        print ("size = " + str(self.size) + " Bytes")

print ("Success class created!")

Success class created!


## Harvest Data from HTTP and Unarchived it <br/> or look in local folder

In [58]:
import urllib.request
import shutil
from os import listdir
import zipfile

#url = "http://ergast.com/downloads/f1db_csv.zip"
#url = url_tb.value
#dataFolder = "f1db_csv/"

def dwnldUrl (_url, folder_name, file_name):            
    
    file_name = '/tmp/' + file_name # file will be downloaded in this location
    # Download the file from 'url' and save it locally under 'file_name':
    with urllib.request.urlopen(_url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)            
    
    # Unarchive data
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall('/tmp/' + folder_name) # creates folder f1db_csv if it doesn't exist
    zip_ref.close()

    listFilePath = []
    
    for f in os.listdir('/tmp/' + folder_name):     
        listFilePath.append('/tmp/' + folder_name + f)
    
    return listFilePath

def getLocalFiles(_path):    
    for f in os.listdir(_path):     
        listFilePath.append(_path + f)
    
    return listFilePath

In [41]:
selSource = widgets.Select(
    options=['Download from URL', 'Local Files'],
    value='Local Files',
    rows=3,
    description='Source:',
    disabled=False
)
selSource

Select(description='Source:', index=1, options=('Download from URL', 'Local Files'), rows=3, value='Local File…

In [74]:
listFilePath = []
if selSource.value == 'Local Files':
    listFilePath = getLocalFiles("/test/HDI/index/")
    for f in listFilePath:     
        print (f)    
    
elif selSource.value == 'Download from URL':
    listFilePath = dwnldUrl("http://ergast.com/downloads/f1db_csv.zip", "f1/", "f1db_csv.zip")
    for f in listFilePath:     
        print (f)

else:
    print ('any')

/test/HDI/index/Human development index (HDI).csv


### Create item objects and append them to a list

In [77]:
import os 
from pyspark import SparkContext

## ITEM objects ###
itemList = []
index = 0

# Ask to introduce schema info if known, else infere schema

for f in listFilePath:  
    df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(f) # note: custom schema should be defined in deduction phase (later)
    content = df.dtypes # list N-tuple <attribute, data type>
    size = os.path.getsize(f) # get file size in Bytes
    ## Instance
    itemList.append(Item(url_tb.value, f, content, size)) # _id, name, content, size
    itemList[index].printInfo()
    print() # Add a blank space between instances
    index+=1

_id  = http://hdr.undp.org/en/indicators/137506#
name  = /test/HDI/index/Human development index (HDI).csv
content  = [('HDI Rank (2017)', 'int'), ('Country', 'string'), ('1990', 'double'), ('1991', 'double'), ('1992', 'double'), ('1993', 'double'), ('1994', 'double'), ('1995', 'double'), ('1996', 'double'), ('1997', 'double'), ('1998', 'double'), ('1999', 'double'), ('2000', 'double'), ('2001', 'double'), ('2002', 'double'), ('2003', 'double'), ('2004', 'double'), ('2005', 'double'), ('2006', 'double'), ('2007', 'double'), ('2008', 'double'), ('2009', 'double'), ('2010', 'double'), ('2011', 'double'), ('2012', 'double'), ('2013', 'double'), ('2014', 'double'), ('2015', 'double'), ('2016', 'double'), ('2017', 'double')]
size = 43731 Bytes



### Create release objects and append them in a list 

In [85]:
#------------------ RELEASE ------------------------------------------------------------------------------------------
# We assume that publication date is the same as the one which the origin server believes the resource was last modified
#conn = urllib.request.urlopen("http://ergast.com/downloads/f1db_csv.zip", timeout=30)
#conn = urllib.request.urlopen(url_tb.value, timeout=30)
last_modified = conn.headers['last-modified']
publicationDate = last_modified

# Get release size <br />
releaseSize = 0
for i in itemList:
    releaseSize = i.size + releaseSize

# Create a list of release objects    
releaseList = []
                           #_id, releaseNum, publicationDate, itemList, size
releaseList.append(Release(url_tb.value, 1, None, itemList, releaseSize)) 
releaseList[0].printInfo()
#--------------------------------------------------------------------------------------------------------------------

_id  = http://hdr.undp.org/en/indicators/137506#
releaseNum  = 1
publicationDate  = None
List of items = [<__main__.Item object at 0x7f20cce7c940>]
size = 43731 Bytes


### Create data collection object
Using release size since both release and collection have the same data <br />
Adding parameter manually, consider extracting them programmatically

In [87]:
#----------- COLLECTION ---------------------------------------------------------------------------------------------
# Ask for database name, provider, licence, author and description
# _id = url maybe change for a hdfs url
                                #_id, name, provider, licence, size, author, description, releaseList
dataCollection = DataCollection(url_tb.value, "ONU", dbprovider_tb.value, dblicense_tb.value, releaseSize, dbauthor_tb.value, dbdesc_tb.value, releaseList) 
dataCollection.printInfo()
#--------------------------------------------------------------------------------------------------------------------

_id  = http://hdr.undp.org/en/indicators/137506#
name  = ONU
provider  = United Nations Development Programme
licence  = Creative Commons Attribution 3.0 IGO
size = 43731 Bytes
author  = Human Development
description  = Add API decription
list of releases = [<__main__.Release object at 0x7f20ccedf518>]


### Serialize data collection objet to JSON

In [88]:
import json

dataCollectionJson = json.dumps(dataCollection, default=lambda x: x.__dict__, indent=3)
dataCollectionJson = json.loads(dataCollectionJson)
print (dataCollectionJson)
print (type(dataCollectionJson))

{'_id': 'http://hdr.undp.org/en/indicators/137506#', 'name': 'ONU', 'provider': 'United Nations Development Programme', 'licence': 'Creative Commons Attribution 3.0 IGO', 'size': 43731, 'author': 'Human Development', 'description': 'Add API decription', 'releaseList': [{'_id': 'http://hdr.undp.org/en/indicators/137506#', 'releaseNum': 1, 'publicationDate': None, 'itemList': [{'_id': 'http://hdr.undp.org/en/indicators/137506#', 'name': '/test/HDI/index/Human development index (HDI).csv', 'content': [['HDI Rank (2017)', 'int'], ['Country', 'string'], ['1990', 'double'], ['1991', 'double'], ['1992', 'double'], ['1993', 'double'], ['1994', 'double'], ['1995', 'double'], ['1996', 'double'], ['1997', 'double'], ['1998', 'double'], ['1999', 'double'], ['2000', 'double'], ['2001', 'double'], ['2002', 'double'], ['2003', 'double'], ['2004', 'double'], ['2005', 'double'], ['2006', 'double'], ['2007', 'double'], ['2008', 'double'], ['2009', 'double'], ['2010', 'double'], ['2011', 'double'], ['201

### Store meta-data file in a MongoDB Atlas Cluster

In [90]:
import json
from pymongo import MongoClient

# Creates a client for the primary sandbox from cluster host cluster0-nlbcx.mongodb.net
client = MongoClient('cluster0-shard-00-00-nlbcx.mongodb.net:27017',
                      username='adminUser',
                      password='xpass',
                      authSource='admin',                      
                      replicaSet='Cluster0-shard-0',
                      ssl=True,
                      retryWrites=True)
db = client.test 
db = client.test_database
db = client['onu_hdi-db']
collection = db.test_collection
collection = db['dataModel-metadata']

pid = collection.insert_one(dataCollectionJson).inserted_id  # this id can replace the url in datacollectionmodel class

import pprint
pprint.pprint(collection.find_one())

client.close()

{'_id': 'http://hdr.undp.org/en/indicators/137506#',
 'author': 'Human Development',
 'description': 'Add API decription',
 'licence': 'Creative Commons Attribution 3.0 IGO',
 'name': 'ONU',
 'provider': 'United Nations Development Programme',
 'releaseList': [{'_id': 'http://hdr.undp.org/en/indicators/137506#',
                  'itemList': [{'_id': 'http://hdr.undp.org/en/indicators/137506#',
                                'content': [['HDI Rank (2017)', 'int'],
                                            ['Country', 'string'],
                                            ['1990', 'double'],
                                            ['1991', 'double'],
                                            ['1992', 'double'],
                                            ['1993', 'double'],
                                            ['1994', 'double'],
                                            ['1995', 'double'],
                                            ['1996', 'double'],
               